In [1]:
import cv2
from utils import display, resizeWithAspectRatio, imageOverlay
import numpy as np
import time

In [34]:
img = cv2.imread('scaled/field.png')
img = cv2.imread("scaled/IMG_1133.jpg")
#img = cv2.imread("scaled/scaleImage2.jpg")
#img = cv2.imread('scaled/field2.png')
img = cv2.imread('scaled/scaleImage7.jpg')

start = time.time()

# resize img for speed
height, width = img.shape[:2]
img = cv2.resize(img, (round(width / 5), round(height / 5)), interpolation=cv2.INTER_AREA)
imgBlurred = cv2.GaussianBlur(img, (5, 5), 0)

imgGray2 = cv2.cvtColor(imgBlurred.copy(), cv2.COLOR_BGR2GRAY)
_, thresh2 = cv2.threshold(imgGray2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

Z = imgBlurred.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 4
ret,label,center=cv2.kmeans(Z,K,None,criteria,1,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

# convert to black and white
imgGray = cv2.cvtColor(res2, cv2.COLOR_BGR2GRAY)
thresh, imgf = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

thresh, imgf = cv2.threshold(imgGray, 180, 255, cv2.THRESH_BINARY)

print(time.time() - start)

display(cv2.threshold(imgGray2, 180, 255, cv2.THRESH_BINARY)[1])
display(thresh2)
display(imgBlurred)
display(res2)
display(imgf)

cv2.waitKey(0)
cv2.destroyAllWindows()

0.06199932098388672


In [25]:
# plot a histogram of an image
for i in cv2.calcHist([wimg],[0],None,[256],[0,256]):
    print(f"{i[0]},")

NameError: name 'wimg' is not defined

In [65]:
type(imgf)

numpy.ndarray

In [77]:
contours, hierarchy = cv2.findContours(imgf, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
cv2.drawContours(imgf, contours, -1, (0,255,0), 10)
display(imgf)

In [80]:
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result

In [83]:
wimg= white_balance(img)

In [17]:
def imageOverlay(background, overlay ,width=0, x=0, y=0):
    """ overlay an image over another """

    dim = (width, width)
    # resize image
    overlay = cv2.resize(overlay, dim, interpolation = cv2.INTER_AREA)
    
    rows,cols,channels = overlay.shape
    overlay=cv2.addWeighted(background[x:x+rows, y:y+cols],0,overlay,1,0)
    background[250:250+rows, 0:0+cols ] = overlay

    return background

In [24]:
overlay = cv2.imread('test_images/target.jpg')
background = cv2.imread('test_images/field2.png')
display(overlay_transparent(background, overlay, x=100, y=100, width=50))

In [7]:
target.shape

(388, 394, 3)

In [8]:
field.shape

(912, 1283, 3)

In [22]:
import cv2
import numpy as np

def overlay_transparent(background, overlay, x, y, width):
    
    dim = (width, width)
    # resize image
    overlay = cv2.resize(overlay, dim, interpolation = cv2.INTER_AREA)

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background

In [25]:
a = np.array([1])

In [26]:
a.tolist()


[1]